In [ ]:
import os
import json
import numpy as np
from Tools.data_process import load_json_data
from Tools.model import BI
import pymc3 as pm

""" Load configuration """
config_path = os.path.join("configs", "model.json")
with open(config_path, "r") as file:
    configs = json.load(file)

data_info = configs["data"]
n_past = data_info["n_past"]
n_future = data_info["n_future"]

""" Load training data """
file_path = os.path.join("data", data_info["filename"])
data_train = load_json_data(file_path, "prediction_train", data_info["json_index"]).drop("forecast", axis=1)

""" Load test data """
data_tests = load_json_data(file_path, "prediction_test", data_info["json_index"])
data_tests_split = {}

""" Extract observed precipitation and runoff """
pq_data = load_json_data(file_path, "prediction_test", ["flood_id", "precipitation", "observed"])
pq_split = {flood_id: group for flood_id, group in pq_data.groupby("flood_id")}

""" Uncertainty quantification """
num_models = 3  # Placeholder value
step_size = n_future
sigmas = np.zeros((num_models, step_size))
weights = np.zeros((num_models, step_size))
conf_intervals, predictions, final_sigmas = [], [], []

for step in range(step_size):
    weight, sigma, final_sigma, pred_value, conf_interval = BI(
        models=[],  # Placeholder
        test_X=[],  # Placeholder
        test_Y=[],  # Placeholder
        modelnames=[],  # Placeholder
        scaler=None,  # Placeholder
        draws=5000,
        tune=2500,
        chains=4,
        confidence=0.95,
        step=step,
    )
    weights[:, step] = weight
    sigmas[:, step] = sigma
    conf_intervals.append(conf_interval)
    predictions.append(pred_value)
    final_sigmas.append(final_sigma)